<a href="https://colab.research.google.com/github/Hardikr23/tensorflow_codes/blob/main/training_with_imageGenerator_with_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5 



In [3]:
%%writefile download.sh

#!/bin/bash
fileid="1onaG42NZft3wCE1WH0GDEbUhu75fedP5"
filename="horse-or-human.zip"
html=`curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${fileid}"`
curl -Lb ./cookie "https://drive.google.com/uc?export=download&`echo ${html}|grep -Po '(confirm=[a-zA-Z0-9\-_]+)'`&id=${fileid}" -o ${filename}

Overwriting download.sh


In [4]:
# NOTE: Please only run this if downloading with gdown did not work.
# This will run the script created above.
!bash download.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  142M  100  142M    0     0   117M      0  0:00:01  0:00:01 --:--:--  208M


In [5]:
import zipfile

# Unzip the dataset
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')
zip_ref.close()

In [6]:
# loading the data file paths
import os

train_horse_imgs = os.path.join('./horse-or-human/horses')
train_human_imgs = os.path.join('./horse-or-human/humans')

In [7]:
train_horse_names = os.listdir(train_horse_imgs)
train_human_names = os.listdir(train_human_imgs)
print(train_human_names)

['human01-14.png', 'human06-15.png', 'human14-18.png', 'human02-27.png', 'human10-24.png', 'human14-19.png', 'human05-06.png', 'human05-05.png', 'human14-08.png', 'human01-17.png', 'human04-05.png', 'human06-01.png', 'human06-03.png', 'human08-14.png', 'human17-13.png', 'human17-25.png', 'human02-30.png', 'human01-20.png', 'human09-02.png', 'human07-02.png', 'human13-22.png', 'human08-18.png', 'human02-13.png', 'human15-02.png', 'human10-09.png', 'human15-20.png', 'human09-21.png', 'human15-09.png', 'human01-05.png', 'human17-18.png', 'human06-20.png', 'human12-11.png', 'human04-13.png', 'human12-29.png', 'human16-21.png', 'human17-26.png', 'human15-06.png', 'human15-22.png', 'human02-12.png', 'human07-19.png', 'human01-28.png', 'human03-17.png', 'human13-10.png', 'human13-23.png', 'human14-24.png', 'human01-06.png', 'human16-27.png', 'human07-04.png', 'human05-16.png', 'human09-29.png', 'human12-13.png', 'human17-02.png', 'human10-23.png', 'human03-26.png', 'human02-06.png', 'human13-

In [8]:
# Now we are ready to build the model

import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential([
                                #  1st convolutional layer
                                 keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
                                 keras.layers.MaxPooling2D(2,2),
                                #  2nd convolutional layer
                                 keras.layers.Conv2D(32, (3,3), activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                 #  3rd convolutional layer
                                 keras.layers.Conv2D(64, (3,3), activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                 #  4th convolutional layer
                                 keras.layers.Conv2D(64, (3,3), activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                 #  5th convolutional layer
                                 keras.layers.Conv2D(64, (3,3), activation='relu'),
                                 keras.layers.MaxPooling2D(2,2),
                                #  Flattening layer
                                 keras.layers.Flatten(),
                                 keras.layers.Dense(512, activation='relu'),
                                 keras.layers.Dense(1, activation='sigmoid')
])

In [9]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [11]:
# preprocessing images for training purpose

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory('./horse-or-human',
                                                    target_size=(300,300),
                                                    batch_size=128,
                                                    class_mode='binary'
                                                    )

Found 1027 images belonging to 2 classes.


In [12]:
history = model.fit(
    train_generator,
    epochs=15,
    verbose=1,
    steps_per_epoch=8
)

Epoch 1/15
8/8 [==============================] - 12s 784ms/step - loss: 0.7475 - accuracy: 0.5373
Epoch 2/15
8/8 [==============================] - 7s 838ms/step - loss: 0.7191 - accuracy: 0.6029
Epoch 3/15
8/8 [==============================] - 7s 834ms/step - loss: 0.5032 - accuracy: 0.8298
Epoch 4/15
8/8 [==============================] - 7s 841ms/step - loss: 1.1434 - accuracy: 0.7208
Epoch 5/15
8/8 [==============================] - 7s 844ms/step - loss: 0.4095 - accuracy: 0.8988
Epoch 6/15
8/8 [==============================] - 7s 853ms/step - loss: 0.2057 - accuracy: 0.9288
Epoch 7/15
8/8 [==============================] - 8s 933ms/step - loss: 0.1555 - accuracy: 0.9404
Epoch 8/15
8/8 [==============================] - 7s 838ms/step - loss: 0.2441 - accuracy: 0.9255
Epoch 9/15
8/8 [==============================] - 7s 807ms/step - loss: 0.1115 - accuracy: 0.9622
Epoch 10/15
8/8 [==============================] - 7s 833ms/step - loss: 0.1489 - accuracy: 0.9410
Epoch 11/15
8/8 [=

In [ ]:
# Download the validation set
!gdown --id 1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U

In [ ]:
# Unzip validation set
import zipfile
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [14]:
# Directory with validation horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with validation human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [15]:
validation_horse_names = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_names[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

VAL SET HORSES: ['horse2-224.png', 'horse5-203.png', 'horse4-501.png', 'horse5-083.png', 'horse1-384.png', 'horse5-235.png', 'horse2-069.png', 'horse4-468.png', 'horse2-412.png', 'horse2-254.png']
VAL SET HUMANS: ['valhuman04-14.png', 'valhuman02-11.png', 'valhuman05-26.png', 'valhuman04-09.png', 'valhuman04-01.png', 'valhuman01-05.png', 'valhuman02-24.png', 'valhuman01-17.png', 'valhuman02-13.png', 'valhuman01-09.png']


In [16]:
validation_datagen = ImageDataGenerator(rescale=1/255)
# Flow validation images in batches of 128 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # This is the source directory for validation images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since you use binary_crossentropy loss, you need binary labels
        class_mode='binary')

Found 256 images belonging to 2 classes.


In [17]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/15
8/8 [==============================] - 10s 1s/step - loss: 0.0858 - accuracy: 0.9766 - val_loss: 1.8595 - val_accuracy: 0.7734
Epoch 2/15
8/8 [==============================] - 9s 1s/step - loss: 0.0558 - accuracy: 0.9844 - val_loss: 1.7349 - val_accuracy: 0.7891
Epoch 3/15
8/8 [==============================] - 9s 1s/step - loss: 0.0240 - accuracy: 0.9944 - val_loss: 2.6759 - val_accuracy: 0.7812
Epoch 4/15
8/8 [==============================] - 9s 1s/step - loss: 0.0121 - accuracy: 0.9967 - val_loss: 2.0082 - val_accuracy: 0.8320
Epoch 5/15
8/8 [==============================] - 8s 1s/step - loss: 0.0674 - accuracy: 0.9789 - val_loss: 1.2435 - val_accuracy: 0.7539
Epoch 6/15
8/8 [==============================] - 8s 1s/step - loss: 0.0817 - accuracy: 0.9611 - val_loss: 2.0949 - val_accuracy: 0.8164
Epoch 7/15
8/8 [==============================] - 8s 1s/step - loss: 0.0066 - accuracy: 1.0000 - val_loss: 2.7475 - val_accuracy: 0.7930
Epoch 8/15
8/8 [========================

In [18]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
    
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

Saving horse_test.png to horse_test (3).png
[1.9382655e-21]
horse_test.png is a horse
